In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import pandas as pd
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import load_fb15k_237, load_yago3_10
from ampligraph.evaluation.protocol import create_mappings, to_idx

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
import time
print(tf.__version__)
assert(tf.__version__.startswith('2.3'))

from ampligraph.datasets import load_fb15k_237, load_fb13, load_fb15k, load_wn11, load_wn18, load_wn18rr, load_yago3_10
from ampligraph.latent_features import ScoringBasedEmbeddingModel

2.3.0


In [2]:
#loss function
def nll(scores_pred, eta):
    scores_neg = scores_pred[1]
    scores_pos = scores_pred[0]

    scores_neg_reshaped = tf.reshape(scores_neg, [eta, tf.shape(scores_pos)[0]])
    neg_exp = tf.exp(scores_neg_reshaped)
    pos_exp = tf.exp(scores_pos)
    softmax_score = pos_exp / (tf.reduce_sum(neg_exp, axis=0) + pos_exp)

    loss = -tf.reduce_sum(tf.math.log(softmax_score))
    return loss

# Train/eval without partition

In [3]:
model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     max_ent_size=14505, 
                                     max_rel_size=237,
                                     scoring_type='TransE')

model.compile(optimizer='adam', loss=nll)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
          batch_size=10000,
          validation_batch_size=100,
          validation_freq=2,
          epochs=10,
          validation_data = '/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt')





Epoch 1/10
28/28 [==============================] - 1s 44ms/step - loss: 17501.9082
Epoch 2/10
28/28 [==============================] - ETA: 0s - loss: 16930.3594
9 triples containing invalid keys skipped!
28/28 [==============================] - 51s 2s/step - loss: 16930.3594 - val_mrr: 0.0965 - val_mr: 1162.6060 - val_hits@1: 0.0035 - val_hits@10: 0.2564
Epoch 3/10
28/28 [==============================] - 1s 44ms/step - loss: 16181.6035
Epoch 4/10
28/28 [==============================] - ETA: 0s - loss: 15331.6602
9 triples containing invalid keys skipped!
28/28 [==============================] - 17s 617ms/step - loss: 15331.6602 - val_mrr: 0.0984 - val_mr: 881.6503 - val_hits@1: 0.0033 - val_hits@10: 0.2622
Epoch 5/10
28/28 [==============================] - 1s 42ms/step - loss: 14492.6719
Epoch 6/10
28/28 [==============================] - ETA: 0s - loss: 13709.9258
9 triples containing invalid keys skipped!
28/28 [==============================] - 17s 617ms/step - loss: 13709.9258

In [4]:

start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', batch_size=400)
end = time.time()
mr_score(ranks), mrr_score(ranks)


28 triples containing invalid keys skipped!
52/52 [==============================] - 41s 793ms/step


(647.0359068627452, 0.08983463806823218)

# Training/eval with partition

In [5]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY
dataset_loader = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt', 
                                 backend=SQLiteAdapter,
                                    batch_size=1000, 
                                    dataset_type='train', 
                                    epochs=10,
                                    use_indexer=True)


partitioner = PARTITION_ALGO_REGISTRY.get('Bucket')(dataset_loader, k=3)

_split: memory before: 0.0Bytes, after: 13.016MB, consumed: 13.016MB; exec time: 210.76s


In [6]:
model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     max_ent_size=9437, 
                                     max_rel_size=237,
                                     scoring_type='TransE')

model.compile(optimizer='adam', loss=nll)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

model.fit(partitioner,
         batch_size=1000, use_partitioning=True,
         epochs=10, callbacks=[checkpoint])

Epoch 1/10
    277/Unknown - 11s 39ms/step - loss: 1674.0751
Epoch 00001: loss improved from inf to 1674.07507, saving model to ./chkpt1
277/277 [==============================] - 12s 42ms/step - loss: 1674.0751
Epoch 2/10
274/277 [============================>.] - ETA: 0s - loss: 1551.6996
Epoch 00002: loss improved from 1674.07507 to 1549.06494, saving model to ./chkpt1
277/277 [==============================] - 9s 33ms/step - loss: 1549.0649
Epoch 3/10
274/277 [============================>.] - ETA: 0s - loss: 1434.5685
Epoch 00003: loss improved from 1549.06494 to 1432.87756, saving model to ./chkpt1
277/277 [==============================] - 10s 35ms/step - loss: 1432.8776
Epoch 4/10
274/277 [============================>.] - ETA: 0s - loss: 1332.1746
Epoch 00004: loss improved from 1432.87756 to 1330.95544, saving model to ./chkpt1
277/277 [==============================] - 10s 35ms/step - loss: 1330.9554
Epoch 5/10
274/277 [============================>.] - ETA: 0s - loss: 1244.

In [7]:

start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=1000)
end = time.time()
mr_score(ranks), mrr_score(ranks)


28 triples containing invalid keys skipped!
21/21 [==============================] - 167s 8s/step


(1269.91475, 0.08386363417991036)